In [28]:
from argopy import IndexFetcher as ArgoIndexFetcher
import pandas as pd
import numpy as np

index_loader = ArgoIndexFetcher()
df=index_loader.region([-180,180,-90.,90.,'2010-01-01','2019-12-31']).to_dataframe()
df.head()

,file,date,longitude,latitude,ocean,date_update,wmo,institution,profiler_type
0,aoml/3900557/profiles/D3900557_136.nc,2010-01-01 00:15:03,-100.681,-22.337,P,2019-01-30 16:35:45,3900557,"AOML, USA","Webb Research, Seabird sensor"
1,csiro/5901692/profiles/D5901692_038.nc,2010-01-01 00:16:51,175.516,-55.421,P,2019-12-17 00:20:04,5901692,"CSIRO, Australia","Webb Research, Seabird sensor"
2,aoml/3900255/profiles/D3900255_184.nc,2010-01-01 00:18:43,-122.220,-20.136,P,2015-03-06 14:50:52,3900255,"AOML, USA","Webb Research, Seabird sensor"
3,aoml/5901303/profiles/D5901303_115.nc,2010-01-01 00:24:57,-119.885,9.402,P,2017-12-18 12:05:14,5901303,"AOML, USA","Webb Research, Seabird sensor"
4,coriolis/6900794/profiles/D6900794_028.nc,2010-01-01 00:27:00,33.022,33.822,A,2015-06-16 09:49:25,6900794,"Ifremer, France","Arvor, Seabird conductivity sensor"


In [29]:
ldat=df.groupby('wmo').last()
#extract months
months=pd.to_datetime(ldat.date.values).month
ldat['month']=months
ldat.head(5)                   

,file,date,longitude,latitude,ocean,date_update,institution,profiler_type,month
wmo,,,,,,,,,
29027,jma/29027/profiles/D29027_480.nc,2010-05-26 22:31:45,128.064,8.950,P,2015-06-21 02:05:14,"JMA, Japan","Webb Research, no conductivity",5
1900046,aoml/1900046/profiles/D1900046_289.nc,2011-03-26 23:14:51,145.435,-42.928,P,2015-02-27 13:42:54,"AOML, USA","Webb Research, Seabird sensor",3
1900050,aoml/1900050/profiles/D1900050_290.nc,2011-03-30 05:51:56,57.223,-32.389,I,2015-02-27 14:55:36,"AOML, USA","Webb Research, Seabird sensor",3
1900053,aoml/1900053/profiles/D1900053_325.nc,2012-04-01 17:06:02,47.779,-41.236,I,2015-02-27 15:26:43,"AOML, USA","Webb Research, Seabird sensor",4
1900054,aoml/1900054/profiles/D1900054_331.nc,2012-06-04 05:42:47,24.594,-37.385,I,2015-02-27 15:31:49,"AOML, USA","Webb Research, Seabird sensor",6


In [30]:
# For future plot
month_name=['January','February','March','April','May','June','July','August','September','October','November','December']
count_a=np.zeros_like(month_name,dtype=float)
count_i=np.zeros_like(month_name,dtype=float)
count_p=np.zeros_like(month_name,dtype=float)
i=0

for m in np.unique(ldat.month):
    si=ldat.query('month == '+str(m)).groupby(['ocean']).size()
    for j in range(len(si)):
        if si.index[j]=="A":
            count_a[i] = si['A'];
        elif si.index[j]=="P":
            count_p[i] = si['P'];
        elif si.index[j]=="I":
            count_i[i] = si['I'];
    i+=1
    
d = {'month': month_name, 'Atlantic': count_a, 'Pacific': count_p, 'Indian': count_i, 
     'Season': ['Winter','Winter','Winter','Spring','Spring','Spring','Summer','Summer','Summer','Autumn','Autumn','Autumn']}
df = pd.DataFrame(data=d)

df

,month,Atlantic,Pacific,Indian,Season
0,January,235.0,350.0,126.0,Winter
1,February,192.0,275.0,139.0,Winter
2,March,270.0,344.0,120.0,Winter
3,April,209.0,353.0,104.0,Spring
4,May,237.0,347.0,134.0,Spring
5,June,215.0,305.0,108.0,Spring
6,July,200.0,310.0,111.0,Summer
7,August,215.0,309.0,140.0,Summer
8,September,194.0,316.0,104.0,Summer
9,October,236.0,332.0,125.0,Autumn


In [33]:
from bokeh.plotting import figure, show
from bokeh.models import Label
from collections import OrderedDict
from bokeh.io import output_notebook

ocean_color = OrderedDict([("Atlantic","#428bca"),("Pacific", "#2e4045"),("Indian",  "#d9534f"  )])
season_color = OrderedDict([("Winter", "#bae1ff"),("Spring", "#96ceb4"),("Summer", "#fff68f"),("Autumn", "#e69584")])

width = 1000
height = 1000
inner_radius = 90
outer_radius = 300

minr = np.log10(1.)
maxr = np.log10(2000)
a = (outer_radius - inner_radius) / (maxr - minr)
b = outer_radius - a * maxr

def rad(mic):
    return a * np.log10(mic) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="", x_axis_type=None, y_axis_type=None, 
           x_range=(-420, 420), y_range=(-420, 420),min_border=0, outline_line_color="black",
           background_fill_color="#f1f1f1")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [season_color[Season] for Season in df.Season]
p.annular_wedge(0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.Atlantic),-big_angle+angles+5*small_angle,
                -big_angle+angles+6*small_angle,color=ocean_color['Atlantic'])
p.annular_wedge(0, 0, inner_radius, rad(df.Pacific),-big_angle+angles+3*small_angle, 
                -big_angle+angles+4*small_angle, color=ocean_color['Pacific'])
p.annular_wedge(0, 0, inner_radius, rad(df.Indian), -big_angle+angles+1*small_angle,
                -big_angle+angles+2*small_angle, color=ocean_color['Indian'])

# circular axes and lables
labels = [1,10,100,1000]
radii = rad(labels)
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii, [str(r) for r in labels], text_font_size="10pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-5, outer_radius+5, -big_angle+angles, -big_angle+angles, color="black")

# DAC labels
xr = (radii[-1]+50)*np.cos(np.array(-big_angle/2 + angles))
yr = (radii[-1]+50)*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.month, angle=label_angle, text_font_size="9pt", text_align="center", text_baseline="middle")

# OK, these hand drawn legends are pretty clunky, will be improved in future release
p.circle([-60, -60, 30, 30], [-360, -380, -360, -380], color=list(season_color.values()), radius=7,line_color='black')
p.text([-45, -45, 45, 45], [-360, -380, -360, -380], text=[gr for gr in season_color.keys()],
       text_font_size="10pt", text_align="left", text_baseline="middle")
p.rect([-30, -30, -30], [18, 0, -18], width=30, height=13, color=list(ocean_color.values()))
p.text([-5, -5, -5], [18, 0, -18], text=list(ocean_color), text_font_size="10pt", text_align="left", text_baseline="middle")

#TITLE
mytext = Label(x=0, y=350, text='Number of argo floats deployed per month & ocean in 2010-2019',text_font_size="12pt", text_align="center", text_baseline="middle")
p.add_layout(mytext)

output_notebook()

show(p)

Loading BokehJS ...